In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle

from naslib.utils import get_dataset_api
from naslib import search_spaces
from naslib.search_spaces.core import Metric
import nasbench301

from nas import _REPO_ROOT

## NAS-Bench-201

In [18]:
nasbench201_api = get_dataset_api("nasbench201", "cifar10")
nasbench201_api

AssertionError: Could not find /home/tomaz/git/Politecnico/Extra/AI-Tech-Lab/nas/test-venv/lib/python3.12/site-packages/naslib/data/nb201_cifar10_full_training.pickle. Please download nb201_cifar10_full_training.pickle from https://drive.google.com/drive/folders/1rwmkqyij3I24zn5GSO6fGv2mzdEfPIEa

In [3]:
architecture = search_spaces.NasBench201SearchSpace()

In [4]:
help(architecture)

Help on NasBench201SearchSpace in module naslib.search_spaces.nasbench201.graph object:

class NasBench201SearchSpace(naslib.search_spaces.core.graph.Graph)
 |  NasBench201SearchSpace(n_classes=10, in_channels=3)
 |
 |  Implementation of the nasbench 201 search space.
 |  It also has an interface to the tabular benchmark of nasbench 201.
 |
 |  Method resolution order:
 |      NasBench201SearchSpace
 |      naslib.search_spaces.core.graph.Graph
 |      torch.nn.modules.module.Module
 |      networkx.classes.digraph.DiGraph
 |      networkx.classes.graph.Graph
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  __init__(self, n_classes=10, in_channels=3)
 |      Initialise a graph. The edges are automatically filled with an EdgeData object
 |      which defines the default operation as Identity. The default combination operation
 |      is set as sum.
 |
 |      Note:
 |          When inheriting form `Graph` note that `__init__()` cannot take any parameters.
 |          This is 

In [5]:
architecture.sample_random_architecture(nasbench201_api)

In [6]:
architecture.query(Metric.RAW, "cifar10", dataset_api=nasbench201_api)

{'cifar10-valid': {'train_losses': [1.9997784596252441,
   1.8044536679840089,
   1.7075754795074463,
   1.6332524340438843,
   1.575655322227478,
   1.5359442371368408,
   1.504173353843689,
   1.467329408416748,
   1.44943655670166,
   1.416760523300171,
   1.402323856163025,
   1.3875704836654663,
   1.3631556325912475,
   1.3452378133392333,
   1.32651785987854,
   1.3105469038772584,
   1.300771376953125,
   1.2804509231185912,
   1.2786731009292602,
   1.2698152198791504,
   1.2402537381362915,
   1.236302218093872,
   1.227023156967163,
   1.2247301602554321,
   1.1942415172576903,
   1.1943177279281616,
   1.17625702293396,
   1.1761633183288573,
   1.1708551595687866,
   1.1416660580825806,
   1.135431934928894,
   1.1496392851257324,
   1.119645326271057,
   1.1080058557891845,
   1.1149090364456178,
   1.0972071747970582,
   1.0900950685119628,
   1.094381051750183,
   1.0792382767105102,
   1.0928624363327026,
   1.0622588133239745,
   1.0501374659729004,
   1.0483271730804

In [7]:
neighbourhood = architecture.get_nbhd(nasbench201_api)
len(neighbourhood)

24

In [8]:
architecture.get_op_indices()

[4, 0, 1, 3, 3, 1]

In [9]:
architecture.mutate(architecture)

AssertionError: An architecture has already been assigned to this instance of NasBench201SearchSpace. Instantiate a new instance to be able to sample a new model or set a new architecture.

In [ ]:
child = search_spaces.NasBench201SearchSpace()
child.mutate(architecture)
child.get_op_indices()

[3, 4, 1, 3, 4, 0]

## NAS-Bench-301

In [3]:
NASBENCH301_DOWNLOAD_DIR = _REPO_ROOT / "models" / "surrogates"

NASBENCH301_MODEL_DIR = NASBENCH301_DOWNLOAD_DIR / "nb_models_1.0"
NASBENCH301_PERFORMANCE_DIR = NASBENCH301_MODEL_DIR / "xgb_v1.0"
NASBENCH301_TIME_DIR = NASBENCH301_MODEL_DIR / "lgb_runtime_v1.0"

NASBENCH301_BENCHMARK_PATH = NASBENCH301_DOWNLOAD_DIR / "nb301_full_training.pickle"

print(NASBENCH301_DOWNLOAD_DIR)
print(NASBENCH301_MODEL_DIR)
print(NASBENCH301_PERFORMANCE_DIR)
print(NASBENCH301_TIME_DIR)
print(NASBENCH301_BENCHMARK_PATH)

/home/tomaz/git/Politecnico/Extra/AI-Tech-Lab/nas/models/surrogates
/home/tomaz/git/Politecnico/Extra/AI-Tech-Lab/nas/models/surrogates/nb_models_1.0
/home/tomaz/git/Politecnico/Extra/AI-Tech-Lab/nas/models/surrogates/nb_models_1.0/xgb_v1.0
/home/tomaz/git/Politecnico/Extra/AI-Tech-Lab/nas/models/surrogates/nb_models_1.0/lgb_runtime_v1.0
/home/tomaz/git/Politecnico/Extra/AI-Tech-Lab/nas/models/surrogates/nb301_full_training.pickle


In [4]:
nasbench301.download_models(
    version="1.0", delete_zip=True, download_dir=str(NASBENCH301_DOWNLOAD_DIR)
)

Models 1.0 already at /home/tomaz/git/Politecnico/Extra/AI-Tech-Lab/nas/models/surrogates/nb_models_1.0


In [ ]:
performance_model = nasbench301.load_ensemble(NASBENCH301_PERFORMANCE_DIR)
runtime_model = nasbench301.load_ensemble(NASBENCH301_TIME_DIR)

nb301_model = [performance_model, runtime_model]
with NASBENCH301_BENCHMARK_PATH.open("rb") as f:
    nb301_data = pickle.load(f)
    nb301_arches = list(nb301_data.keys())

nasbench301_api = {
    "nb301_data": nb301_data,
    "nb301_arches": nb301_arches,
    "nb301_model": nb301_model,
}

So called "labelled" architectures are those from the training set of the surrogate model, that is, those for which training has actually been performed.

In [6]:
nb301_architecture = search_spaces.NasBench301SearchSpace()
nb301_architecture.sample_random_architecture(nasbench301_api, load_labeled=True)
nb301_architecture

Graph makrograph-0.9668268, scope None, 13 nodes

In [7]:
nb301_architecture.query(Metric.VAL_ACCURACY, dataset_api=nasbench301_api)

93.01000213623047

In [34]:
nb301_architecture.query(Metric.TRAIN_TIME, dataset_api=nasbench301_api)

9230.06887

In [10]:
nb301_neighbourhood = nb301_architecture.get_nbhd(nasbench301_api)
len(nb301_neighbourhood)

120

In [36]:
type(nb301_neighbourhood[0].arch)

naslib.search_spaces.nasbench301.graph.NasBench301SearchSpace

In [17]:
nb301_architecture.get_compact()

(((0, 0), (1, 3), (1, 4), (2, 3), (0, 1), (3, 5), (2, 2), (3, 3)),
 ((0, 3), (1, 6), (0, 6), (1, 4), (0, 1), (2, 0), (1, 2), (4, 3)))

In [37]:
nb301_child = search_spaces.NasBench301SearchSpace()
nb301_child.mutate(nb301_architecture)
nb301_child.get_compact()

(((0, 0), (1, 3), (1, 4), (2, 3), (0, 1), (3, 5), (2, 2), (3, 3)),
 ((0, 3), (1, 6), (2, 6), (1, 4), (1, 1), (2, 0), (1, 2), (4, 3)))